In [69]:
#Loading the Required Libraries Required to frame the model
import json
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn import model_selection,preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn import linear_model
from xgboost import XGBClassifier
from pandas_profiling import ProfileReport
!pip3 install pandas_profiling --upgrade
import os

Requirement already up-to-date: pandas_profiling in /usr/local/lib/python3.6/dist-packages (2.10.0)


LOADING THE DATASET \
\
Our Dataset is given in JSON Format i.e. if we talk in normal words, in the form of Dictionary of Dictionaries

Our First Job is to extract the Features and its corresponding Data. After Extraction we will do the visualization and EDA on the data set inorder to figure out the Correlations between the features and How we need to do the feature Engineering

In [70]:
f_data=open('/content/ml_work')
data=json.load(f_data)
n=len(data)
n

1000

EXTRACTING THE FEATURES TO FORM A DATAFRAME \
(Next Step for Feature Engineering) 

In [71]:
#The Respective Arrays to Store the Datas for the particular values
description=[]
fresh=[]
secure=[]
stale=[]
safety=[]
host=[]
content_type=[]
content_length=[]
title=[]
user_agent=[]

for i in range(0,n):
  description.append(data[i]['req']['body']['note']['desc'])
  fresh.append(data[i]['req']['fresh'])
  secure.append(data[i]['req']['secure'])
  stale.append(data[i]['req']['stale'])
  host.append(data[i]['req']['headers']['host'])
  content_type.append(data[i]['req']['headers']['content-type'])
  content_length.append(data[i]['req']['headers']['content-length'])
  title.append(data[i]['req']['body']['note']['title'])
  user_agent.append(data[i]['req']['headers']['user-agent'])
  safety.append(data[i]['isSafe'])

df_data=pd.DataFrame()
df_data['Title']=title
df_data['Description']=description
df_data["Fresh"]=fresh
df_data['Secure']=secure
df_data['Host']=host
df_data['Stale']=stale
df_data['Content_Type']=content_type
df_data['Content_Length']=content_length
df_data['User_Agent']=user_agent
df_data['Access_Granted']=safety

##Our DataFrame is formed which will be used for the ML Model

Finally, Our Dataframe is prepared which is going to be used for our Machine Learning Model.Lets take a look at the dataframe and go to the feature engineering and the EDA Portion of the Datasets.

In [72]:
df_data.head(10)
df_data.to_csv('Access_Data.csv')

Visualization and EDA for the Datasets

> For Each Feature and We will do the required Feature Engineering as well



In [73]:
df_data.describe()

,Title,Description,Fresh,Secure,Host,Stale,Content_Type,Content_Length,User_Agent,Access_Granted
count,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000
unique,994,750,1,1,1,1,1,1,1,2
top,Michael Smith,note title <title onPropertyChange title onPro...,False,False,example.com,True,application/json,70,insomnia/2020.4.2,True
freq,3,36,1000,1000,1000,1000,1000,1000,1000,572


After Going through the Data, We get to know that All the Features except for the Descriptions are Gaussian and Irrelevant inorder to make the prediction


So We will Drop all our Gaussian Feature 

In [75]:
gaussian=['Title','Fresh','Secure','Host','Stale','Content_Type','Content_Length','User_Agent']
df_data.drop(gaussian,axis=1,inplace=True)
df_data.head()

,Description,Access_Granted
0,Top recognize eat. Fact whom spend area thing ...,True
1,As possible American many prepare four strong....,True
2,Tuesday Notes or 2 like 2 XSP Class,False
3,State third represent energy campaign not forg...,True
4,Us enjoy since. Time identify image position o...,False


In [76]:

##Randomly Shuffle the Data
df_data.sample(frac=1).reset_index(drop=True)
df_data.Access_Granted.replace([False,True],[0,1],inplace=True)
df_data.Access_Granted.value_counts()

1    572
0    428
Name: Access_Granted, dtype: int64

#PREPARE DATASETS

Prepare the Dataset for 

In [77]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
Y=df_data.Access_Granted.values
X=df_data.Description.values
stopword=stopwords.words('english')
wnl=WordNetLemmatizer()
for i in range(0,len(X)):
  #Tokenize the Wordds
  X[i]=word_tokenize(X[i])
  #X[i]=wnl.lemmatize(' '.join(X[i]))
  #Removing the Stopwords 
  X[i]=[j for j in X[i] if j not in stopword and j not in string.punctuation]
  X[i]=' '.join(X[i])


xtrain,xtest,ytrain,ytest=model_selection.train_test_split(X,Y,test_size=0.25,random_state=None)
cv=CountVectorizer(tokenizer=word_tokenize,token_pattern=None)
xtrain=cv.fit_transform(xtrain)
xtest=cv.transform(xtest)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#MODEL 1
#Logistic Regression

In [78]:
lr=linear_model.LogisticRegression(C=0.3)
lr.fit(xtrain,ytrain)
ytrain_pred=lr.predict(xtrain)
ytest_pred=lr.predict(xtest)

accuracy1=metrics.accuracy_score(ytrain,ytrain_pred)
accuracy2=metrics.accuracy_score(ytest,ytest_pred)
f1_1=metrics.f1_score(ytrain,ytrain_pred)
f1_2=metrics.f1_score(ytest,ytest_pred)

print('Train and Test Accuracy =',accuracy1*100,accuracy2*100)
print('F1_Scores =',f1_1*100,f1_2*100)

Train and Test Accuracy = 97.06666666666666 83.6
F1_Scores = 97.57174392935983 86.1952861952862


#The Logistic Regression Model has quite Good Accuracy

It has an Accuracy of 96.8% on the Training Set(75% of the Total Dataset)

It has an Accuracy of 82% on the test set which is quite decent

The F1-Scores are Quite decent as Well

#MODEL NO - 2
#SVM

Although We have Good Accuracy on Logistic Regression. Let's try to make a Model of SVMs on the Dataset

In [79]:
nb=svm.SVC()
#Lets do the Hyperparameter tuning onto the Dataset
params={'C':np.arange(0.05,1.05,0.05),'gamma':np.arange(0.1,1.1,0.1),'kernel':['linear','rbf']}
clf=model_selection.GridSearchCV(estimator=nb,param_grid=params,verbose=10,n_jobs=1,scoring='accuracy',cv=5)

clf.fit(xtrain,ytrain)
ytrain_pred=clf.predict(xtrain)
ytest_pred=clf.predict(xtest)

print(clf.best_score_)
print(clf.best_estimator_)

Fitting 5 folds for each of 400 candidates, totalling 2000 fits
[CV] C=0.05, gamma=0.1, kernel=linear ................................
[CV] .... C=0.05, gamma=0.1, kernel=linear, score=0.820, total=   0.1s
[CV] C=0.05, gamma=0.1, kernel=linear ................................
[CV] .... C=0.05, gamma=0.1, kernel=linear, score=0.860, total=   0.1s
[CV] C=0.05, gamma=0.1, kernel=linear ................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


[CV] .... C=0.05, gamma=0.1, kernel=linear, score=0.820, total=   0.1s
[CV] C=0.05, gamma=0.1, kernel=linear ................................
[CV] .... C=0.05, gamma=0.1, kernel=linear, score=0.840, total=   0.1s
[CV] C=0.05, gamma=0.1, kernel=linear ................................
[CV] .... C=0.05, gamma=0.1, kernel=linear, score=0.807, total=   0.1s
[CV] C=0.05, gamma=0.1, kernel=rbf ...................................


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.3s remaining:    0.0s


[CV] ....... C=0.05, gamma=0.1, kernel=rbf, score=0.760, total=   0.1s
[CV] C=0.05, gamma=0.1, kernel=rbf ...................................
[CV] ....... C=0.05, gamma=0.1, kernel=rbf, score=0.787, total=   0.1s
[CV] C=0.05, gamma=0.1, kernel=rbf ...................................


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.6s remaining:    0.0s


[CV] ....... C=0.05, gamma=0.1, kernel=rbf, score=0.760, total=   0.1s
[CV] C=0.05, gamma=0.1, kernel=rbf ...................................
[CV] ....... C=0.05, gamma=0.1, kernel=rbf, score=0.740, total=   0.1s
[CV] C=0.05, gamma=0.1, kernel=rbf ...................................
[CV] ....... C=0.05, gamma=0.1, kernel=rbf, score=0.727, total=   0.1s
[CV] C=0.05, gamma=0.2, kernel=linear ................................

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.8s remaining:    0.0s



[CV] .... C=0.05, gamma=0.2, kernel=linear, score=0.820, total=   0.1s
[CV] C=0.05, gamma=0.2, kernel=linear ................................
[CV] .... C=0.05, gamma=0.2, kernel=linear, score=0.860, total=   0.1s
[CV] C=0.05, gamma=0.2, kernel=linear ................................
[CV] .... C=0.05, gamma=0.2, kernel=linear, score=0.820, total=   0.1s
[CV] C=0.05, gamma=0.2, kernel=linear ................................
[CV] .... C=0.05, gamma=0.2, kernel=linear, score=0.840, total=   0.1s
[CV] C=0.05, gamma=0.2, kernel=linear ................................
[CV] .... C=0.05, gamma=0.2, kernel=linear, score=0.807, total=   0.1s
[CV] C=0.05, gamma=0.2, kernel=rbf ...................................
[CV] ....... C=0.05, gamma=0.2, kernel=rbf, score=0.760, total=   0.1s
[CV] C=0.05, gamma=0.2, kernel=rbf ...................................
[CV] ....... C=0.05, gamma=0.2, kernel=rbf, score=0.660, total=   0.1s
[CV] C=0.05, gamma=0.2, kernel=rbf ...................................
[CV] 

[Parallel(n_jobs=1)]: Done 2000 out of 2000 | elapsed:  2.9min finished


We get Quite A Good Result Using the SVM Classifier

The Highlighting Point in this Model is the Tuning of the Hyperparameters of the Particular Classifier
We tuned the gamma, Inverse of Regularization parameter(lambda) which is known as C, and the Type of Kernel We want to use


In [80]:
## Without Tuning SVMs
sv=svm.SVC()
sv.fit(xtrain,ytrain)

ytrain_pred=sv.predict(xtrain)
ytest_pred=sv.predict(xtest)

accuracy1=metrics.accuracy_score(ytrain,ytrain_pred)
accuracy2=metrics.accuracy_score(ytest,ytest_pred)
f1_1=metrics.f1_score(ytrain,ytrain_pred)
f1_2=metrics.f1_score(ytest,ytest_pred)

print('Model Statistics Without Hyperparameter Tuning')
print('Train and Test Accuracy =',accuracy1*100,accuracy2*100)
print('F1_Scores =',f1_1*100,f1_2*100)

Model Statistics Without Hyperparameter Tuning
Train and Test Accuracy = 96.0 83.6
F1_Scores = 96.71772428884026 86.37873754152824


#Inference

We got to know that the SVM Model gives better Accuracy and has better F1_Score on the Test_Set more than the Logistic Regression Model


#MODEL 3

#Random Forest Classifier

In [81]:
rf=RandomForestClassifier(n_jobs=-1)
params={'max_depth':np.arange(1,20),'n_estimators':np.arange(10,101,10),'criterion':['gini','entropy']}

clf=model_selection.RandomizedSearchCV(estimator=rf,param_distributions=params,verbose=10,cv=5,n_jobs=1,scoring='accuracy')
clf.fit(xtrain,ytrain)

print(clf.best_estimator_)
print(clf.best_score_)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] n_estimators=50, max_depth=11, criterion=entropy ................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimators=50, max_depth=11, criterion=entropy, score=0.820, total=   1.2s
[CV] n_estimators=50, max_depth=11, criterion=entropy ................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s


[CV]  n_estimators=50, max_depth=11, criterion=entropy, score=0.860, total=   0.3s
[CV] n_estimators=50, max_depth=11, criterion=entropy ................


...

[CV]  n_estimators=100, max_depth=11, criterion=gini, score=0.840, total=   0.4s
[CV] n_estimators=100, max_depth=11, criterion=gini ..................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    3.6s remaining:    0.0s


[CV]  n_estimators=100, max_depth=11, criterion=gini, score=0.807, total=   0.3s
[CV] n_estimators=90, max_depth=2, criterion=gini ....................
[CV]  n_estimators=90, max_depth=2, criterion=gini, score=0.760, total=   0.4s
[CV] n_estimators=90, max_depth=2, criterion=gini ....................
[CV]  n_estimators=90, max_depth=2, criterion=gini, score=0.780, total=   0.4s
[CV] n_estimators=90, max_depth=2, criterion=gini ....................
[CV]  n_estimators=90, max_depth=2, criterion=gini, score=0.813, total=   0.4s
[CV] n_estimators=90, max_depth=2, criterion=gini ....................
[CV]  n_estimators=90, max_depth=2, criterion=gini, score=0.793, total=   0.4s
[CV] n_estimators=90, max_depth=2, criterion=gini ....................
[CV]  n_estimators=90, max_depth=2, criterion=gini, score=0.780, total=   0.4s
[CV] n_estimators=10, max_depth=10, criterion=gini ...................
[CV]  n_estimators=10, max_depth=10, criterion=gini, score=0.820, total=   0.1s
[CV] n_estimators=

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   13.8s finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=11, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)
0.8293333333333333


In [82]:
## Without Tuning
rf=RandomForestClassifier()
rf.fit(xtrain,ytrain)

ytrain_pred=rf.predict(xtrain)
ytest_pred=rf.predict(xtest)

accuracy1=metrics.accuracy_score(ytrain,ytrain_pred)
accuracy2=metrics.accuracy_score(ytest,ytest_pred)
f1_1=metrics.f1_score(ytrain,ytrain_pred)
f1_2=metrics.f1_score(ytest,ytest_pred)

print('Model Statistics Without Hyperparameter Tuning')
print('Train and Test Accuracy =',accuracy1*100,accuracy2*100)
print('F1_Scores =',f1_1*100,f1_2*100)

Model Statistics Without Hyperparameter Tuning
Train and Test Accuracy = 100.0 83.6
F1_Scores = 100.0 86.37873754152824
